In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import plotly.graph_objects as go
import collections
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
url = "https://www.seek.co.nz/jobs?classification=6281%2C1223%2C1209%2C6008"
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data,'html.parser')
response

<Response [200]>

In [3]:
job_no = 0
jobs_dict = {}
while True:
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    jobs = soup.find_all('article',{'class':'_2m3Is-x'})
    
    for job in jobs:
        title = job.find('a',{'data-automation':'jobTitle'}).text
        location_tag = job.find('a',{'data-automation':'jobLocation'})
        location = location_tag.text if location_tag else 'N/A'
        link = 'https://www.seek.co.nz' + job.find('a',{'class':'_2iNL7wI'}).get('href')
        job_response = requests.get(link)
        #print(job_response)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data, 'html.parser')
        job_description = job_soup.find('div',{'data-automation':'jobDescription'}).text
        Date_posted = job_soup.find('dd',{'data-automation':'job-detail-date'}).text
        work_type_tag = job_soup.find('dd',{'data-automation':'job-detail-work-type'})
        work_type = work_type_tag.text if location_tag else 'N/A'
        job_no += 1
        jobs_dict[job_no] = [Date_posted, title, location, work_type, link, job_description]
        #print(title)
        #print(Date_posted)
        #print(location)
        #print(work_type)
        #print(link)
        #print(job_description)
    url_tag = soup.find('a',{'class':'bHpQ-bp'})   
    if url_tag:
        url = 'https://www.seek.co.nz' + url_tag.get('href')
        #print(url)
    else:
        break
print('Total jobs',job_no)    

Total jobs 4150


In [4]:
job_dataset = pd.DataFrame.from_dict(jobs_dict,orient = 'index', columns = ['Date','Title','Location','Type','link','Job Description'])
job_dataset.head()

,Date,Title,Location,Type,link,Job Description
1,23 Jan 2020,Systems Engineer - Auckland,Auckland,Full Time,https://www.seek.co.nz/job/40813144?type=promo...,"At CommArc, we've been helping Kiwi businesses..."
2,29 Jan 2020,Senior Microsoft Dynamics 365 Business Central...,Wellington,Full Time,https://www.seek.co.nz/job/40835921?type=promo...,We are growing and we need you! We are seeking...
3,18 Feb 2020,Service Desk Analyst,Auckland,Full Time,https://www.seek.co.nz/job/40981549?type=stand...,Service Desk Analyst Rewarding and diverse pe...
4,20 Feb 2020,Level 1&2 Technical Support,Auckland,Full Time,https://www.seek.co.nz/job/41003172?type=stand...,"\n<!--#VideoJobAd,.videoembed{display:block;he..."
5,20 Feb 2020,Service Desk Analyst,Auckland,Contract/Temp,https://www.seek.co.nz/job/41004091?type=stand...,The Opportunity We are currently seeking a pro...


In [5]:
job_dataset.to_csv('jobs_unfiltered.csv')

In [6]:
languages = ['PYTHON','C++','R','SAS','POWERBI','TABLEAU','SQL','JAVA','JAVASCRIPT','AWS','HADOOP','JAVASCRIPT','HTML','CSS','C#','EXCEL', 'OFFICE 365',
            'AZURE','DJANGO','REACT', 'HTML5', 'ASP.NET', 'GIT', 'LINUX','DOCKER', 'KUBERNETES', 'PHP', 'ANGULAR', 'JQUERY','API']

disciplines = ['AI', 'BI']

roles = ['DATA ANALYST', 'DATA SCIENTIST', 'DATA ENGINEER','BI DEVELOPER','WEB DEVELOPER', 'SOFTWARE DEVELOPER']

def word_find(line,words):
    return list(set(line.strip().split()) & set(words)) if set(line.strip().split()) & set(words) else 'N/A'

In [7]:
#format = lambda x: word_find(x.upper(),languages)
job_dataset['languages'] = job_dataset['Job Description'].apply(lambda x: word_find(x.upper(),languages))
job_dataset['disciplines'] = job_dataset['Job Description'].apply(lambda x: word_find(x.upper(),disciplines))
job_dataset['roles'] = job_dataset['Job Description'].apply(lambda x: word_find(x.upper(),roles))
job_dataset.head()

,Date,Title,Location,Type,link,Job Description,languages,disciplines,roles
1,23 Jan 2020,Systems Engineer - Auckland,Auckland,Full Time,https://www.seek.co.nz/job/40813144?type=promo...,"At CommArc, we've been helping Kiwi businesses...",N/A,N/A,N/A
2,29 Jan 2020,Senior Microsoft Dynamics 365 Business Central...,Wellington,Full Time,https://www.seek.co.nz/job/40835921?type=promo...,We are growing and we need you! We are seeking...,N/A,[BI],N/A
3,18 Feb 2020,Service Desk Analyst,Auckland,Full Time,https://www.seek.co.nz/job/40981549?type=stand...,Service Desk Analyst Rewarding and diverse pe...,N/A,N/A,N/A
4,20 Feb 2020,Level 1&2 Technical Support,Auckland,Full Time,https://www.seek.co.nz/job/41003172?type=stand...,"\n<!--#VideoJobAd,.videoembed{display:block;he...",N/A,N/A,N/A
5,20 Feb 2020,Service Desk Analyst,Auckland,Contract/Temp,https://www.seek.co.nz/job/41004091?type=stand...,The Opportunity We are currently seeking a pro...,N/A,N/A,N/A


In [8]:
jobs = job_dataset.drop(['link', 'Job Description'], axis=1)
jobs['Date'] = pd.to_datetime(jobs['Date'])
jobs.shape

(4150, 7)

In [9]:
jobs['Title'][jobs['Title'].str.contains('Entry')]

2011    Service Desk - Level 1 Support | Experienced &...
2480    Service Desk - Level 1 Support | Experienced &...
Name: Title, dtype: object

In [14]:
lang_dict = {}
discip_dict = {}
role_dict = {}
def find_item(word,lst):
    result = any((elem) == word for elem in lst)
    if result: 
        return 1
    else:
        return 0 

for language in languages:
    jobs['is'+ language] = jobs['languages'].apply(lambda x: find_item(language,x))
    lang_dict[language] = jobs['is' + language][jobs['is' + language]==1].count()
for discipline in disciplines:
    jobs['is'+ discipline] = jobs['disciplines'].apply(lambda x: find_item(discipline,x))
    discip_dict[discipline] = jobs['is' + discipline][jobs['is' + discipline]==1].count()
for role in roles:
    jobs['is'+ role] = jobs['disciplines'].apply(lambda x: find_item(role,x))
    role_dict[role] = jobs['is' + role][jobs['is' + role]==1].count()    
jobs.head()

,Date,Title,Location,Type,languages,disciplines,roles,isPYTHON,isC++,isR,...,isJQUERY,isAPI,isAI,isBI,isDATA ANALYST,isDATA SCIENTIST,isDATA ENGINEER,isBI DEVELOPER,isWEB DEVELOPER,isSOFTWARE DEVELOPER
1,2020-01-23,Systems Engineer - Auckland,Auckland,Full Time,N/A,N/A,N/A,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-29,Senior Microsoft Dynamics 365 Business Central...,Wellington,Full Time,N/A,[BI],N/A,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2020-02-18,Service Desk Analyst,Auckland,Full Time,N/A,N/A,N/A,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-02-20,Level 1&2 Technical Support,Auckland,Full Time,N/A,N/A,N/A,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2020-02-20,Service Desk Analyst,Auckland,Contract/Temp,N/A,N/A,N/A,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
lang_dict_sorted = sorted(lang_dict.items(), key=lambda kv: kv[1])
lang_dict_sorted = collections.OrderedDict(lang_dict_sorted)
x = list(lang_dict_sorted.keys())
y = list(lang_dict_sorted.values())
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )])

fig.show()

In [16]:
discip_dict

{'AI': 38, 'BI': 108}

In [17]:
discip_dict_sorted = sorted(discip_dict.items(), key=lambda kv: kv[1])
discip_dict_sorted = collections.OrderedDict(discip_dict_sorted)
x = list(discip_dict_sorted.keys())
y = list(discip_dict_sorted.values())
# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            textposition='auto',
        )])

fig.show()

In [ ]:
jobs.to_csv('jobs.csv')